In [1]:
from bs4 import BeautifulSoup
import requests as rt
import pandas as pd

In [2]:
url = 'https://josaa.admissions.nic.in/applicant/seatmatrix/OpeningClosingRankArchieve.aspx'
params = {
    #"ctl00$ContentPlaceHolder1$ddlroundno": 
    "ctl00$ContentPlaceHolder1$ddlInstype": "ALL",
    "ctl00$ContentPlaceHolder1$ddlInstitute": "ALL",
    "ctl00$ContentPlaceHolder1$ddlBranch": "ALL",
    "ctl00$ContentPlaceHolder1$ddlSeatType": "ALL",
    "ctl00$ContentPlaceHolder1$btnSubmit": "Submit"
}

In [3]:
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
rounds = ['1', '2', '3', '4', '5', '6', '7']

In [4]:
def scrape(Year, Round):   
    with rt.Session() as s:
        R = s.get(url)
        data = {}

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlYear"] = Year
        R = s.post(url, data=data)

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlroundno"] = Round
        R = s.post(url, data=data)

        for key, value in params.items():
            data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
            data[key] = value
            R = s.post(url, data=data)
        #print(R.text)
        #print(data)

    table = BeautifulSoup(R.text, 'lxml').find(id = 'ctl00_ContentPlaceHolder1_GridView1')
    df = pd.read_html(table.prettify())[0]
    df.dropna(inplace = True, how="all")
    
    df['Year'] = Year
    df['Round'] = Round
    #df['Opening Rank'] = df['Opening Rank'].astype(int)
    #df['Closing Rank'] = df['Closing Rank'].astype(int)
    
    return df

In [10]:
df = scrape('2022', '6')
df

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
0,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Gender-Neutral,9193,13624,2022,6
1,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Female-only (including Supernumerary),16138,21674,2022,6
2,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,EWS,Gender-Neutral,1907,2171,2022,6
3,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,EWS,Female-only (including Supernumerary),3334,3334,2022,6
4,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OBC-NCL,Gender-Neutral,4094,4615,2022,6
...,...,...,...,...,...,...,...,...,...
9727,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,OPEN,Gender-Neutral,56910,76463,2022,6
9728,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,EWS,Gender-Neutral,11263,11413,2022,6
9729,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,OBC-NCL,Gender-Neutral,22569,24055,2022,6
9730,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,SC,Gender-Neutral,12164,12435,2022,6


In [6]:
#df['Opening Rank']= df['Opening Rank'].str.replace('P', '')
open = df.loc[(df['Seat Type']=='OPEN')& (df['Gender']=='Gender-Neutral') & (df.Quota=='AI')]
open['Opening Rank'] = open['Opening Rank'].astype(int)
open['Closing Rank'] = open['Closing Rank'].astype(int)
open

/tmp/ipykernel_7440/630540295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  open['Opening Rank'] = open['Opening Rank'].astype(int)
/tmp/ipykernel_7440/630540295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  open['Closing Rank'] = open['Closing Rank'].astype(int)


,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
0,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Gender-Neutral,8816,11238,2020,6
11,Indian Institute of Technology Bhubaneswar,Civil Engineering and M. Tech. in Structural E...,AI,OPEN,Gender-Neutral,11424,12002,2020,6
19,Indian Institute of Technology Bhubaneswar,Civil Engineering and M.Tech in Transportation...,AI,OPEN,Gender-Neutral,11501,12492,2020,6
27,Indian Institute of Technology Bhubaneswar,Civil Engineering and M.Tech. in Environmental...,AI,OPEN,Gender-Neutral,11146,12734,2020,6
35,Indian Institute of Technology Bhubaneswar,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,1319,2251,2020,6
...,...,...,...,...,...,...,...,...,...
8970,lndian Institute of Food Processing Technology...,"Food Technology (4 Years, Bachelor of Technology)",AI,OPEN,Gender-Neutral,39832,63112,2020,6
8976,North Eastern Regional Institute of Science an...,"Agricultural Engineering (4 Years, Bachelor of...",AI,OPEN,Gender-Neutral,64236,64236,2020,6
8978,North Eastern Regional Institute of Science an...,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Gender-Neutral,48291,48291,2020,6
8984,North Eastern Regional Institute of Science an...,"Electrical Engineering (4 Years, Bachelor of T...",AI,OPEN,Gender-Neutral,52857,52857,2020,6


In [7]:
open = open.sort_values('Opening Rank')
open.head(40)

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
186,Indian Institute of Technology Bombay,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,2,66,2020,6
8650,"School of Planning & Architecture, New Delhi","Architecture (5 Years, Bachelor of Architecture)",AI,OPEN,Gender-Neutral,3,230,2020,6
212,Indian Institute of Technology Bombay,"Electrical Engineering (4 Years, Bachelor of T...",AI,OPEN,Gender-Neutral,25,298,2020,6
434,Indian Institute of Technology Delhi,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,31,105,2020,6
8657,"School of Planning & Architecture, New Delhi","Planning (4 Years, Bachelor of Planning)",AI,OPEN,Gender-Neutral,46,479,2020,6
1414,Indian Institute of Technology Madras,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,49,158,2020,6
8662,School of Planning & Architecture: Vijayawada,"Architecture (5 Years, Bachelor of Architecture)",AI,OPEN,Gender-Neutral,68,694,2020,6
450,Indian Institute of Technology Delhi,"Computer Science and Engineering (5 Years, Bac...",AI,OPEN,Gender-Neutral,108,189,2020,6
1255,Indian Institute of Technology Kanpur,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,111,239,2020,6
144,Indian Institute of Technology Bombay,"BS in Mathematics (4 Years, Bachelor of Science)",AI,OPEN,Gender-Neutral,128,1567,2020,6


In [41]:
df['Academic Program Name'].str.split('Planning ')

0       [Civil Engineering (4 Years, Bachelor of Techn...
1       [Civil Engineering (4 Years, Bachelor of Techn...
2       [Civil Engineering (4 Years, Bachelor of Techn...
3       [Civil Engineering (4 Years, Bachelor of Techn...
4       [Civil Engineering (4 Years, Bachelor of Techn...
                              ...                        
9727    [Electronics and Communication Engineering (Av...
9728    [Electronics and Communication Engineering (Av...
9729    [Electronics and Communication Engineering (Av...
9730    [Electronics and Communication Engineering (Av...
9731    [Electronics and Communication Engineering (Av...
Name: Academic Program Name, Length: 9732, dtype: object

In [9]:
df.columns

Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object')

In [46]:
df.Quota.unique()

array(['AI', 'HS', 'OS', 'GO', 'JK', 'LA'], dtype=object)

In [47]:
df['Seat Type'].unique()

array(['OPEN', 'EWS', 'OBC-NCL', 'SC', 'ST', 'OPEN (PwD)', 'EWS (PwD)',
       'OBC-NCL (PwD)', 'SC (PwD)', 'ST (PwD)'], dtype=object)

In [48]:
df['Gender'].unique()

array(['Gender-Neutral', 'Female-only (including Supernumerary)'],
      dtype=object)

In [49]:
df.to_csv('2022_all_6.csv')

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
0,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Gender-Neutral,9193,13624,2022,6
1,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Female-only (including Supernumerary),16138,21674,2022,6
2,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,EWS,Gender-Neutral,1907,2171,2022,6
3,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,EWS,Female-only (including Supernumerary),3334,3334,2022,6
4,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OBC-NCL,Gender-Neutral,4094,4615,2022,6
...,...,...,...,...,...,...,...,...,...
9727,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,OPEN,Gender-Neutral,56910,76463,2022,6
9728,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,EWS,Gender-Neutral,11263,11413,2022,6
9729,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,OBC-NCL,Gender-Neutral,22569,24055,2022,6
9730,Central University of Jammu,Electronics and Communication Engineering (Avi...,AI,SC,Gender-Neutral,12164,12435,2022,6


In [53]:
df['Institute'].unique()

array(['Indian Institute of Technology Bhubaneswar',
       'Indian Institute of Technology Bombay',
       'Indian Institute of Technology Mandi',
       'Indian Institute of Technology Delhi',
       'Indian Institute of Technology Indore',
       'Indian Institute of Technology Kharagpur',
       'Indian Institute of Technology Hyderabad',
       'Indian Institute of Technology Jodhpur',
       'Indian Institute of Technology Kanpur',
       'Indian Institute of Technology Madras',
       'Indian Institute of Technology Gandhinagar',
       'Indian Institute of Technology Patna',
       'Indian Institute of Technology Roorkee',
       'Indian Institute of Technology (ISM) Dhanbad',
       'Indian Institute of Technology Ropar',
       'Indian Institute of Technology (BHU) Varanasi',
       'Indian Institute of Technology Guwahati',
       'Indian Institute of Technology Bhilai',
       'Indian Institute of Technology Goa',
       'Indian Institute of Technology Palakkad',
       'In